In [ ]:
from nltk.corpus import sentiwordnet as swn 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
import string
import re
import pymongo
#from pymongo import MongoClient 
from nltk.tokenize import TweetTokenizer 
import nltk
from nltk.tokenize import PunktSentenceTokenizer
import collections
import operator
from nltk.util import ngrams
import datetime 
import numpy as np
from nltk.corpus import sentiwordnet as swn 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
import string
import re
import pymongo
#from pymongo import MongoClient 
from nltk.tokenize import TweetTokenizer 
import nltk
from nltk.tokenize import PunktSentenceTokenizer
import collections
import operator
from nltk.util import ngrams
import json

client = pymongo.MongoClient('mongodb://TwitterIPO?authSource=TwitterIPO')

stop_words = list(stopwords.words('english'))

def clean_tweet(t):

    t = t.replace('RT ','')
    t = t.replace('$','')
    t = t.replace('#','')
    
    for word in t:
        if word.startswith(("@")) == True:
            t = t.replace(word,'')
            
    t = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''','',t)

    t = t.replace(',','')
    t = t.replace('.','')
    t = t.replace(';','')
    t = t.replace("'",'')
    t = t.replace('"','')
    t = t.replace('...','')
    t = t.replace('…','')
    t = t.replace('’','')
    t = t.replace('/','')
    t = t.replace('/','')
    t = t.replace('-->','')
    t = t.replace('(','')
    t = t.replace('[','')
    t = t.replace(']','')
    t = t.replace(')','')
    t = t.replace(':','')
    t = t.replace('-','')
    t = t.replace('!','')
    t = t.replace('|','')
    t = t.replace('?','')
    t = t.replace('>','')
    t = t.replace('*','')
    return(t.lower())

allowed_word_types = ["J","R","V"]

cursor=client.TwitterIPO.ProcessedTweets.find({ "lang": "en" } )
tweetDataFrame =  pd.DataFrame(list(cursor))
tweetDataFrame

wordcountdic={}
tweetmessages=pd.unique(tweetDataFrame[['tweet_id', 'text','company']].values)

positiveWord= client.TwitterIPO.PositiveWords.find({},{"Word":1}).sort([("Word",pymongo.ASCENDING)])
negativeWord= client.TwitterIPO.NegativeWords.find({},{"Word":1}).sort([("Word",pymongo.ASCENDING)])
positiveWordDF =  pd.DataFrame(list(positiveWord))
negativeWordDF =  pd.DataFrame(list(negativeWord))

tknz = TweetTokenizer()
tokenizetweet=()

wordfrequencycursor=client.TwitterIPO.WordFrequencies.find({ },{"Word":1})
wordfrequencydf =  pd.DataFrame(list(wordfrequencycursor))
newcolumn=wordfrequencydf['Word'].tolist()

wordOccurences=dict.fromkeys(newcolumn,0)

tweetDataFrame['date_minus_time'] = tweetDataFrame["datepy"].apply( lambda dt : datetime.datetime(year=dt.year, month=dt.month, day=dt.day)) 
result = tweetDataFrame[['company','date_minus_time']].groupby(['company','date_minus_time']).count()
datedf=result.reset_index()
companyList=tweetDataFrame.groupby(['company']).count().reset_index()


for row in companyList.iterrows():
    
    company = row[1][0]
    print("start company ",company)
    cursor3=client.TwitterIPO.ProcessedTweets.find({ "company": company} )
    companytweetDataFrame =  pd.DataFrame(list(cursor3))
    companytweetDataFrame['date_minus_time'] = companytweetDataFrame["datepy"].apply( lambda dt : datetime.datetime(year=dt.year, month=dt.month, day=dt.day)) 
   
    listofdate= companytweetDataFrame[['date_minus_time']].groupby(['date_minus_time']).count().reset_index() #datedf[datedf.company==company]

    for date in listofdate.iterrows():
        currentdate=date[1][0]
        #print(type(currentdate))
        #print("start date ")
        alltweetoncurrentdate=companytweetDataFrame[(companytweetDataFrame.date_minus_time== currentdate)]
        for textinthatday in alltweetoncurrentdate.iterrows():
            text=textinthatday[1][6]
            date=textinthatday[1][4]
            cleantweet= clean_tweet(text)
            listoftoken= tknz.tokenize(cleantweet)
            for currenttoken in listoftoken:
                if currenttoken in newcolumn:
                    numberoftokent=list(client.TwitterIPO.CompanyTweetWordVector.find({ "ipo_company": company,"price_date":currentdate},{currenttoken:1}))
  
                    originalvalue=numberoftokent[0][currenttoken]
                    if((positiveWordDF['Word']==currenttoken).any()):
                        newvalue= originalvalue+1
                        #print(newvalue)
                        client.TwitterIPO.CompanyTweetWordVector.update_one({"ipo_company": company,"price_date": currentdate} , {"$set": {currenttoken: newvalue}})
                    if((negativeWordDF['Word']==currenttoken).any()):
                        newvalue= originalvalue-1
                        #print(newvalue)
                        client.TwitterIPO.CompanyTweetWordVector.update_one({ "ipo_company": company,"price_date":currentdate} , {"$set": {currenttoken: newvalue}})
                        





